In [1]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
sys.path.append(".")
# os.environ["MAX_JOBS"] = "100"

import torch
torch.set_printoptions(linewidth=200, threshold=100000)
torch.cuda.is_available()

True

In [2]:
from src.vlstm_fwbw_v1.interface import vlstm_fwbw_torch_obw, vlstm_fwbw_cuda
from src.vlstm_fwbw_v1.interface import vlstm_fwbw_torch_autogradbw

INCLUDE: ['/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/torch/csrc/api/include', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/TH', '/home/max/miniconda3/envs/xlstmpt220cu121/lib/python3.11/site-packages/torch/include/THC', '/home/max/miniconda3/envs/xlstmpt220cu121/include']
/home/max/myrepos/vlstm_cuda/src
/home/max/cpplibs/libtorch/lib:/usr/local/cuda-12.3/lib64:
/home/max/miniconda3/envs/xlstmpt220cu121/lib


Using /home/max/.cache/torch_extensions/py311_cu121 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/max/.cache/torch_extensions/py311_cu121/vlstm_fwbw_v1/build.ninja...
Building extension module vlstm_fwbw_v1...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module vlstm_fwbw_v1...


## CUDA vLSTM forward backward

In [3]:
S = 8 #32 #16 #8 # seq len
B = 1 # batch size
NH = 1 # num heads
DH = 8 # dim per head
DTYPE = torch.float32 #bfloat16 #float32
DEVICE = torch.device("cuda:0")

In [4]:
# create qkv, inputgates, forgetgates 
torch.manual_seed(0)
# fixed:
# qs = torch.arange((B*NH*S*DH), device=DEVICE, dtype=DTYPE).reshape((B, NH, S, DH)) / 10.
# ks = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.
# vs = torch.ones((B, NH, S, DH), device=DEVICE, dtype=DTYPE) / 100.

# random: 
qs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
ks = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
vs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)
# igs = (1. + torch.arange((B * NH * S), device=DEVICE, dtype=DTYPE)).reshape(B, NH, S, 1) / 10.
# igs = torch.zeros((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
igs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE) #/ 10.
# fgs = torch.ones((B, NH, S, 1), device=DEVICE, dtype=DTYPE)
fgs = torch.randn((B, NH, S, 1), device=DEVICE, dtype=DTYPE)

# dHs = torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

### match through autograd

In [5]:
# now we need to apply the groupnorm directly afterwards
from vlstm_formulations.ln import MultiHeadLayerNorm

mh_layernorm = MultiHeadLayerNorm(ndim=NH*DH, eps=1e-6).to(device=DEVICE, dtype=DTYPE)
mh_layernorm.weight, mh_layernorm.bias

(Parameter containing:
 tensor([0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0', requires_grad=True),
 None)

In [6]:
offset = 3.* torch.randn((B, NH, S, DH), device=DEVICE, dtype=DTYPE)

In [7]:
fgs_pt_ag = fgs.clone().detach().requires_grad_(True)
igs_pt_ag = igs.clone().detach().requires_grad_(True)
qs_pt_ag = qs.clone().detach().requires_grad_(True)
ks_pt_ag = ks.clone().detach().requires_grad_(True)
vs_pt_ag = vs.clone().detach().requires_grad_(True)

In [8]:
# pytorch obw
hs_pt_ag, n_pt_ag, m_pt_ag = vlstm_fwbw_torch_obw( #vlstm_fwbw_torch_autogradbw( #vlstm_fwbw_torch_obw(
    queries=qs_pt_ag,
    keys=ks_pt_ag,
    values=vs_pt_ag,
    igate_preact=igs_pt_ag,
    fgate_preact=fgs_pt_ag,
)
hs_pt_ag_scaled = mh_layernorm(hs_pt_ag)
# hs_pt_ag_scaled.sum().backward()
((hs_pt_ag_scaled+offset)**2).sum().backward()

In [9]:
fgs_cu_ag = fgs.clone().detach().requires_grad_(True)
igs_cu_ag = igs.clone().detach().requires_grad_(True)
qs_cu_ag = qs.clone().detach().requires_grad_(True)
ks_cu_ag = ks.clone().detach().requires_grad_(True)
vs_cu_ag = vs.clone().detach().requires_grad_(True)

In [10]:
# pytorch autograd
# hs_cu_ag, n_cu_ag, m_cu_ag = vlstm_fwbw_torch_autogradbw(
#     queries=qs_cu_ag,
#     keys=ks_cu_ag,
#     values=vs_cu_ag,
#     igate_preact=igs_cu_ag,
#     fgate_preact=fgs_cu_ag,
# )
# hs_cu_ag_scaled = mh_layernorm(hs_cu_ag)
# hs_cu_ag_scaled.sum().backward()

In [11]:
# cuda kernel
hs_cu_ag, n_cu_ag, m_cu_ag, matD_cu_ag = vlstm_fwbw_cuda(
    mat_Q=qs_cu_ag,
    mat_K=ks_cu_ag,
    mat_V=vs_cu_ag,
    vec_igp=igs_cu_ag,
    vec_fgp=fgs_cu_ag,
)
hs_cu_ag_scaled = mh_layernorm(hs_cu_ag)
# hs_cu_ag_scaled.sum().backward()
((hs_cu_ag_scaled+offset)**2).sum().backward()

before kernel dispatch - float32!
B: 1, NH: 1, S: 8, DH: 8
blocksxy: 1-1, threadsxy: 4-4, shared_mem in bytes: 5664
In FW-Kernel: gdim.x: 1, gdim.y: 1, gdim.z: 1, bdim.x: 4, bdim.y: 4
In FW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4
before kernel dispatch - float32!
B: 1, NH: 1, S: 8, DH: 8
blocksxy: 1-2, threadsxy: 4-4, shared_mem in bytes: 7648
In BW-Kernel: gdim.x: 1, gdim.y: 2, gdim.z: 1, bdim.x: 4, bdim.y: 4
In BW-Kernel: QtileDim: 8, KVtileDim: 8, TblockDim:4


In [12]:
FW_RTOL = 1e-10
FW_ATOL = 1e-4
BW_RTOL = FW_ATOL
BW_ATOL = FW_ATOL
print(f"fw hs match: {torch.allclose(hs_cu_ag, hs_pt_ag, rtol=FW_RTOL, atol=FW_ATOL)}")
print(f"fw hs scaled match: {torch.allclose(hs_cu_ag_scaled, hs_pt_ag_scaled, rtol=FW_RTOL, atol=FW_ATOL)}")
print(f"fw n match: {torch.allclose(n_cu_ag, n_pt_ag, rtol=FW_RTOL, atol=FW_ATOL)}")
print(f"fw m match: {torch.allclose(m_cu_ag, m_pt_ag, rtol=FW_RTOL, atol=FW_ATOL)}")
# print(f"fw D match: {torch.allclose((matD_cu_ag - matD_pt_ag).tril(), torch.zeros_like((matD_cu)), rtol=FW_RTOL, atol=FW_ATOL)}")

print(f"delta Q match: {torch.allclose(qs_cu_ag.grad, qs_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")
print(f"delta K match: {torch.allclose(ks_cu_ag.grad, ks_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")
print(f"delta V match: {torch.allclose(vs_cu_ag.grad, vs_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")
print(f"delta Igate match: {torch.allclose(igs_cu_ag.grad, igs_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")
print(f"delta Fgate match: {torch.allclose(fgs_cu_ag.grad, fgs_pt_ag.grad, rtol=BW_RTOL, atol=BW_ATOL)}")

fw hs match: True
fw hs scaled match: True
fw n match: True
fw m match: True
delta Q match: True
delta K match: True
delta V match: True
delta Igate match: True
delta Fgate match: True


In [13]:
m_cu_ag, m_pt_ag

(tensor([[[[ 0.3516],
           [-0.0763],
           [-0.5240],
           [ 0.3096],
           [-0.3649],
           [ 0.7268],
           [ 0.1638],
           [ 0.4116]]]], device='cuda:0', grad_fn=<vLSTMParallelFwBwCudaBackward>),
 tensor([[[[ 0.3516],
           [-0.0763],
           [-0.5240],
           [ 0.3096],
           [-0.3649],
           [ 0.7268],
           [ 0.1638],
           [ 0.4116]]]], device='cuda:0', grad_fn=<vLSTMParallelFwBwWithGroupNormBackward>))

In [14]:
qs_cu_ag.grad - qs_pt_ag.grad

tensor([[[[ 2.8096e-07, -8.5808e-07,  1.4354e-06, -1.1420e-06,  2.1442e-06,  1.3481e-06,  2.0145e-07, -1.4615e-06],
          [ 1.7881e-07, -3.5763e-07,  7.1526e-07, -6.1840e-07,  1.0654e-06,  6.8545e-07,  5.9605e-08, -7.3016e-07],
          [ 1.1921e-07,  9.5367e-07, -9.5367e-07,  2.3842e-07,  9.5367e-07, -4.7684e-07,  2.3842e-07, -4.7684e-07],
          [-2.3842e-07, -2.3842e-07,  5.9605e-07,  0.0000e+00,  2.3842e-07,  2.3842e-07,  1.3113e-06,  3.5763e-07],
          [ 3.9637e-06,  7.6294e-06, -1.2875e-05,  6.6757e-06, -8.5831e-06, -5.7220e-06, -2.1458e-05, -3.3379e-06],
          [-1.6689e-06, -1.4007e-06,  9.5367e-07, -1.4603e-06,  7.6294e-06,  7.1526e-07,  1.0878e-06,  1.4305e-06],
          [ 0.0000e+00,  1.1921e-07, -1.1921e-07,  2.9802e-08,  4.7684e-07, -4.7684e-07, -8.9407e-08,  1.1921e-07],
          [ 7.6294e-06,  1.9073e-06, -7.6294e-06,  4.7684e-07,  7.6294e-06, -5.7220e-06, -7.6294e-06,  0.0000e+00]]]], device='cuda:0')

In [15]:
torch.abs(qs_cu_ag.grad - qs_pt_ag.grad).max()

tensor(2.1458e-05, device='cuda:0')

In [16]:
torch.abs(ks_cu_ag.grad - ks_pt_ag.grad).max()

tensor(1.6212e-05, device='cuda:0')

In [17]:
torch.abs(ks_cu_ag.grad - ks_pt_ag.grad).max()

tensor(1.6212e-05, device='cuda:0')

In [18]:
torch.abs(vs_cu_ag.grad - vs_pt_ag.grad).max()

tensor(4.7684e-06, device='cuda:0')